In [41]:
import sqlite3
import pandas as pd
import paramiko
import os

# IMPORTS

# Configuración del SFTP

In [43]:
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get variables from environment
SFTP_HOST = os.getenv('SFTP_HOST')
SFTP_PORT = int(os.getenv('SFTP_PORT'))
SFTP_USER = os.getenv('SFTP_USER')
SFTP_PRIVATE_KEY = os.getenv('SFTP_PRIVATE_KEY')
RUTA = os.getenv('RUTA')

# Generate file name and paths
time = pd.Timestamp.now()
file = 'reporte_consolidado-' + time.strftime('%Y-%m-%d-%H_%M') + '.csv'

LOCAL_PATH = RUTA + file
print(LOCAL_PATH)

REMOTE_PATH = '/data/sftp/reporte_consolidado-' + file

C:/Users/alarc/Desktop/DataMining/ETL/reporte_consolidado-2024-11-27-10_28.csv


# Proceso ETL

In [29]:
print("== Paso 1: Extracción de datos ==")

# Conectar a SQLite y extraer datos
db_path = 'comercio_ventas.db'
conn = sqlite3.connect(db_path)

== Paso 1: Extracción de datos ==


# Extracción de datos de SQLite

In [30]:
comercio_df = pd.read_sql_query("SELECT * FROM Comercio", conn)
producto_df = pd.read_sql_query("SELECT * FROM Producto", conn)
transaccion_df = pd.read_sql_query("SELECT * FROM Transaccion", conn)
detalle_transaccion_df = pd.read_sql_query("SELECT * FROM Detalle_Transaccion", conn)

# Cerrar la conexión a la base de datos
conn.close()

# Imprimir los encabezados para verificar los datos extraídos

In [31]:
print("Encabezados de Comercio:")
print(comercio_df.head(), "\n")

print("Encabezados de Producto:")
print(producto_df.head(), "\n")

print("Encabezados de Transacción:")
print(transaccion_df.head(), "\n")

print("Encabezados de Detalle Transacción:")
print(detalle_transaccion_df.head(), "\n")

Encabezados de Comercio:
   id_comercio        RUC codigo_comercio nombre_comercio    direccion  \
0            1  205500001            C001      Comercio 1  Dirección X   
1            2  205500002            C002      Comercio 2  Dirección X   
2            3  205500003            C003      Comercio 3  Dirección X   
3            4  205500004            C004      Comercio 4  Dirección X   
4            5  205500005            C005      Comercio 5  Dirección X   

    telefono                email fecha_registro ciudad    distrito provincia  \
0  999999999  correo1@example.com     2024-11-14   Lima  San Isidro      Lima   
1  999999999  correo2@example.com     2024-11-14   Lima  San Isidro      Lima   
2  999999999  correo3@example.com     2024-11-14   Lima  San Isidro      Lima   
3  999999999  correo4@example.com     2024-11-14   Lima  San Isidro      Lima   
4  999999999  correo5@example.com     2024-11-14   Lima  San Isidro      Lima   

  departamento tipo_comercio     sector  in

# Extracción de datos externos (archivos CSV)

In [32]:
tarifas_bancarias = pd.read_csv('tarifas_bancarias.csv')  # Contiene banco, tipo_tarjeta y comisión
dias_festivos = pd.read_csv('dias_festivos.csv')  # Contiene fecha y descripción del día festivo

# Imprimir encabezados de archivos CSV
print("Encabezados de Tarifas Bancarias:")
print(tarifas_bancarias.head(), "\n")

print("Encabezados de Días Festivos:")
print(dias_festivos.head(), "\n")


Encabezados de Tarifas Bancarias:
        banco tipo_tarjeta  comision
0         BCP       Débito     0.015
1         BCP      Crédito     0.020
2   Interbank       Débito     0.012
3   Interbank      Crédito     0.018
4  Scotiabank       Débito     0.014 

Encabezados de Días Festivos:
        fecha              descripcion
0  2024-01-01                Año Nuevo
1  2024-04-18             Jueves Santo
2  2024-04-19            Viernes Santo
3  2024-05-01          Día del Trabajo
4  2024-07-28  Día de la Independencia 



# Paso 2: Transformación de datos

In [33]:
print("== Paso 2: Transformación de datos ==")

# 1. Unir tablas de SQLite
transaccion_completa = pd.merge(transaccion_df, detalle_transaccion_df, left_on='id_transaccion', right_on='id_transaccion')
transaccion_completa = pd.merge(transaccion_completa, comercio_df, left_on='id_comercio', right_on='id_comercio')
transaccion_completa = pd.merge(transaccion_completa, producto_df, left_on='id_producto', right_on='id_producto', suffixes=('_comercio', '_producto'))

# Imprimir encabezados después de la unión
print("Encabezados después de unir Transacción y Detalle:")
print(transaccion_completa.head(), "\n")

# 2. Agregar información de tarifas bancarias
transaccion_completa = pd.merge(
    transaccion_completa, 
    tarifas_bancarias, 
    how='left', 
    left_on=['banco_emisor', 'tipo_tarjeta'], 
    right_on=['banco', 'tipo_tarjeta']
)

# Imprimir encabezados después de agregar tarifas bancarias
print("Encabezados después de agregar tarifas bancarias:")
print(transaccion_completa.head(), "\n")

== Paso 2: Transformación de datos ==
Encabezados después de unir Transacción y Detalle:
   id_transaccion  id_comercio_comercio           fecha_transaccion  \
0               1                     1  2024-10-05 00:26:43.323858   
1               1                     1  2024-10-05 00:26:43.323858   
2               2                     2  2024-07-02 00:26:43.337182   
3               2                     2  2024-07-02 00:26:43.337182   
4               3                     5  2024-05-30 00:26:43.337182   

  tipo_operacion id_operacion      estado   monto moneda  id_producto  \
0          Venta     OP000001  Completado   55.23  Soles            7   
1          Venta     OP000001  Completado   55.23  Soles            7   
2          Venta     OP000002  Completado   26.80  Soles            5   
3          Venta     OP000002  Completado   26.80  Soles            5   
4          Venta     OP000003   Rechazado  165.60  Soles            5   

  motivo_venta_observada  ...     sector ingr

# 3. Agregar información de días festivos

In [34]:
transaccion_completa['fecha_transaccion'] = pd.to_datetime(transaccion_completa['fecha_transaccion'])
dias_festivos['fecha'] = pd.to_datetime(dias_festivos['fecha'])
transaccion_completa = pd.merge(
    transaccion_completa, 
    dias_festivos, 
    how='left', 
    left_on='fecha_transaccion', 
    right_on='fecha'
)
transaccion_completa['es_dia_festivo'] = ~transaccion_completa['descripcion'].isna()

# Imprimir encabezados después de agregar días festivos
print("Encabezados después de agregar días festivos:")
print(transaccion_completa.head(), "\n")

Encabezados después de agregar días festivos:
   id_transaccion  id_comercio_comercio          fecha_transaccion  \
0               1                     1 2024-10-05 00:26:43.323858   
1               1                     1 2024-10-05 00:26:43.323858   
2               2                     2 2024-07-02 00:26:43.337182   
3               2                     2 2024-07-02 00:26:43.337182   
4               3                     5 2024-05-30 00:26:43.337182   

  tipo_operacion id_operacion      estado   monto moneda  id_producto  \
0          Venta     OP000001  Completado   55.23  Soles            7   
1          Venta     OP000001  Completado   55.23  Soles            7   
2          Venta     OP000002  Completado   26.80  Soles            5   
3          Venta     OP000002  Completado   26.80  Soles            5   
4          Venta     OP000003   Rechazado  165.60  Soles            5   

  motivo_venta_observada  ... nivel_lealtad id_comercio_producto  \
0      Sin observaciones  

# 4. Calcular comisiones bancarias aplicadas

In [35]:
transaccion_completa['comision_aplicada'] = transaccion_completa['monto'] * transaccion_completa['comision']


# 5. Agregar columnas adicionales

In [36]:

transaccion_completa['DayOfMonth'] = transaccion_completa['fecha_transaccion'].dt.day

# Imprimir encabezados después de agregar cálculos adicionales
print("Encabezados después de cálculos adicionales:")
print(transaccion_completa.head(), "\n")

# Paso 3: Carga de datos
print("== Paso 3: Carga de datos ==")

# Guardar el resultado consolidado
transaccion_completa.to_csv(LOCAL_PATH, index=False)
print(f"Archivo consolidado guardado localmente como {LOCAL_PATH}.\n")

# Subir el archivo al servidor SFTP
try:
    # Verificar si el archivo local existe
    if not os.path.exists(LOCAL_PATH):
        print(f"Error: El archivo '{LOCAL_PATH}' no existe.")
        exit()

    # Crear conexión SFTP
    private_key = paramiko.RSAKey.from_private_key_file(SFTP_PRIVATE_KEY)
    transport = paramiko.Transport((SFTP_HOST, SFTP_PORT))
    transport.connect(username=SFTP_USER, pkey=private_key)
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Subir el archivo
    print(f"Subiendo {LOCAL_PATH} a {REMOTE_PATH}...")
    sftp.put(LOCAL_PATH, REMOTE_PATH)
    print("Archivo subido con éxito.")

    # Cerrar la conexión
    sftp.close()
    transport.close()
except FileNotFoundError as e:
    print(f"Error: Archivo no encontrado. {e}")
except paramiko.SSHException as e:
    print(f"Error SSH: {e}")
except Exception as e:
    print(f"Error al subir el archivo al servidor SFTP: {e}")

Encabezados después de cálculos adicionales:
   id_transaccion  id_comercio_comercio          fecha_transaccion  \
0               1                     1 2024-10-05 00:26:43.323858   
1               1                     1 2024-10-05 00:26:43.323858   
2               2                     2 2024-07-02 00:26:43.337182   
3               2                     2 2024-07-02 00:26:43.337182   
4               3                     5 2024-05-30 00:26:43.337182   

  tipo_operacion id_operacion      estado   monto moneda  id_producto  \
0          Venta     OP000001  Completado   55.23  Soles            7   
1          Venta     OP000001  Completado   55.23  Soles            7   
2          Venta     OP000002  Completado   26.80  Soles            5   
3          Venta     OP000002  Completado   26.80  Soles            5   
4          Venta     OP000003   Rechazado  165.60  Soles            5   

  motivo_venta_observada  ... nombre_producto    categoria precio_unitario  \
0      Sin observ

OSError: Cannot save file into a non-existent directory: 'c:\Users\alarc\Desktop\DataMining\ETL\venv\Scripts;C:\Users\alarc\Desktop\DataMining\ETL\venv\Scripts;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Common Files\Oracle\Java\javapath;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\Git\cmd;C:\Program Files\nodejs\;C:\Program Files\playit_gg\bin\;C:\Program Files\NVIDIA Corporation\NVIDIA app\NvDLISR;C:\Program Files\WireGuard\;C:\Program Files\Docker\Docker\resources\bin;C:\Program Files\dotnet\;C:\Program Files\NVIDIA Corporation\Nsight Compute 2024.3.2\;C:\Users\alarc\Documents\github\PWD\Utils\hashcat-6.2.6;C:\Program Files\PuTTY\;C:\Users\alarc\.console-ninja\.bin;C:\Users\alarc\AppData\Local\Microsoft\WindowsApps;C:\Users\alarc\Ap

# Descargar archivo del servidor SFTP

In [45]:
import paramiko
import os
from datetime import datetime


LOCAL_DIR = os.getenv('RUTA')
REMOTE_DIR = '/data/sftp/'

try:
    # Crear conexión SFTP
    private_key = paramiko.RSAKey.from_private_key_file(SFTP_PRIVATE_KEY)
    transport = paramiko.Transport((SFTP_HOST, SFTP_PORT))
    transport.connect(username=SFTP_USER, pkey=private_key)
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Listar archivos en el directorio remoto
    print(f"Conectando al servidor SFTP en {REMOTE_DIR}...")
    files = sftp.listdir_attr(REMOTE_DIR)
    
    # Buscar el archivo más reciente
    latest_file = None
    latest_time = None
    for file in files:
        if file.filename.endswith('.csv'):  # Solo considerar archivos .csv
            file_time = file.st_mtime  # Tiempo de modificación
            if latest_time is None or file_time > latest_time:
                latest_time = file_time
                latest_file = file

    if latest_file:
        # Descargar el archivo más reciente
        remote_file_path = REMOTE_DIR + latest_file.filename
        local_file_path = os.path.join(LOCAL_DIR, latest_file.filename)
        print(f"Descargando el archivo más reciente: {latest_file.filename}")
        sftp.get(remote_file_path, local_file_path)
        print(f"Archivo descargado con éxito: {local_file_path}")
    else:
        print("No se encontraron archivos .csv en el directorio remoto.")

    # Cerrar la conexión
    sftp.close()
    transport.close()
except FileNotFoundError as e:
    print(f"Error: Archivo no encontrado. {e}")
except paramiko.SSHException as e:
    print(f"Error SSH: {e}")
except Exception as e:
    print(f"Error: {e}")


Conectando al servidor SFTP en /data/sftp/...
Descargando el archivo más reciente: reporte_consolidado-reporte_consolidado-2024-11-26-23_55.csv
Archivo descargado con éxito: C:/Users/alarc/Desktop/DataMining/ETL/reporte_consolidado-reporte_consolidado-2024-11-26-23_55.csv
